# Dataset
We will explore this dataset: https://archive.ics.uci.edu/ml/datasets/EEG+Eye+State#

> All data is from one continuous EEG measurement with the Emotiv EEG Neuroheadset. The duration of the measurement was 117 seconds. The eye state was detected via a camera during the EEG measurement and added later manually to the file after analysing the video frames. '1' indicates the eye-closed and '0' the eye-open state. All values are in chronological order with the first measured value at the top of the data.

In [60]:
%load_ext autoreload
%autoreload 2
from sys import path
import torch
import torch.utils.data.dataloader
import numpy as np
path.append('..')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
import tensorflow as tf
data_dir = "../../data/raw"
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00264/EEG%20Eye%20State.arff"
datapath = tf.keras.utils.get_file(
        "eeg", origin=url, untar=False, cache_dir=data_dir
    )

from scipy.io import arff
data = arff.loadarff(datapath)
    

In [3]:
type(data)

tuple

In [5]:
len(data)

2

In [6]:
type(data[0]) , type (data[1])

(numpy.ndarray, scipy.io.arff._arffread.MetaData)

In [63]:
arrData = data[0]
for i in arrData:
    if(i[14] == b'0' ):
        i[14] = '0'
    else:
        i[14] = '1'

np.append(arrData,[0], axis=0)

TypeError: The DTypes <class 'numpy.dtype[int32]'> and <class 'numpy.dtype[void]'> do not have a common DType. For example they cannot be stored in a single array unless the dtype is `object`.

In [66]:
newdata = data[0].tolist()
type(newdata)

list

In [78]:
npdata = np.array(
    newdata,
    dtype = np.float16
)

npdata 

array([[4.328e+03, 4.010e+03, 4.288e+03, ..., 4.636e+03, 4.392e+03,
        0.000e+00],
       [4.324e+03, 4.004e+03, 4.292e+03, ..., 4.632e+03, 4.384e+03,
        0.000e+00],
       [4.328e+03, 4.006e+03, 4.296e+03, ..., 4.628e+03, 4.388e+03,
        0.000e+00],
       ...,
       [4.276e+03, 3.990e+03, 4.248e+03, ..., 4.592e+03, 4.340e+03,
        1.000e+00],
       [4.284e+03, 3.992e+03, 4.252e+03, ..., 4.596e+03, 4.352e+03,
        1.000e+00],
       [4.288e+03, 3.998e+03, 4.260e+03, ..., 4.596e+03, 4.352e+03,
        1.000e+00]], dtype=float16)

In [49]:
for i, element in enumerate(data[0]):
    if element[14] == b'0':
        data[0][i][14] = '0'
    else:
        data[0][i][14] = '1'
data[0]

array([(4329.23, 4009.23, 4289.23, 4148.21, 4350.26, 4586.15, 4096.92, 4641.03, 4222.05, 4238.46, 4211.28, 4280.51, 4635.9 , 4393.85, b'0'),
       (4324.62, 4004.62, 4293.85, 4148.72, 4342.05, 4586.67, 4097.44, 4638.97, 4210.77, 4226.67, 4207.69, 4279.49, 4632.82, 4384.1 , b'0'),
       (4327.69, 4006.67, 4295.38, 4156.41, 4336.92, 4583.59, 4096.92, 4630.26, 4207.69, 4222.05, 4206.67, 4282.05, 4628.72, 4389.23, b'0'),
       ...,
       (4277.44, 3990.77, 4246.67, 4113.85, 4333.33, 4615.38, 4072.82, 4623.59, 4193.33, 4212.82, 4160.51, 4257.95, 4591.79, 4339.49, b'1'),
       (4284.62, 3991.79, 4251.28, 4122.05, 4334.36, 4616.41, 4080.51, 4628.72, 4200.  , 4220.  , 4165.64, 4267.18, 4596.41, 4350.77, b'1'),
       (4287.69, 3997.44, 4260.  , 4121.03, 4333.33, 4616.41, 4088.72, 4638.46, 4212.31, 4226.67, 4167.69, 4274.36, 4597.95, 4350.77, b'1')],
      dtype=[('AF3', '<f8'), ('F7', '<f8'), ('F3', '<f8'), ('FC5', '<f8'), ('T7', '<f8'), ('P7', '<f8'), ('O1', '<f8'), ('O2', '<f8'), ('P8',

In [44]:
type(newdata[0][14])


tuple

In [121]:
import torch
import torch.utils.data.dataloader
import numpy as np

class BaseListDataset:
    def __init__(self, data: list , horizon: int):
        self.data = data
        self.process_data()
        self.size = len(data)
        self.horizon = horizon
    
    def __len__(self) -> int:
        return self.size
    
    def __getitem__(self, idx) -> tuple:
        x = self.data[:-self.horizon,:]
        y = self.data[-self.horizon:,:]
        return x, y

In [125]:
class EEGListDataset(BaseListDataset):
    def process_data(self) -> None:
        data = np.array(self.data, dtype=np.float16)
        self.data = torch.Tensor(data)

In [127]:
dsNew = EEGListDataset(data=data[0].tolist(),horizon=1)
x,y = dsNew[0]
x

tensor([[4.3280e+03, 4.0100e+03, 4.2880e+03,  ..., 4.6360e+03, 4.3920e+03,
         0.0000e+00],
        [4.3240e+03, 4.0040e+03, 4.2920e+03,  ..., 4.6320e+03, 4.3840e+03,
         0.0000e+00],
        [4.3280e+03, 4.0060e+03, 4.2960e+03,  ..., 4.6280e+03, 4.3880e+03,
         0.0000e+00],
        ...,
        [4.2760e+03, 3.9920e+03, 4.2440e+03,  ..., 4.5920e+03, 4.3320e+03,
         1.0000e+00],
        [4.2760e+03, 3.9900e+03, 4.2480e+03,  ..., 4.5920e+03, 4.3400e+03,
         1.0000e+00],
        [4.2840e+03, 3.9920e+03, 4.2520e+03,  ..., 4.5960e+03, 4.3520e+03,
         1.0000e+00]])